<a href="https://colab.research.google.com/github/Annettteee/annette-colab-projects/blob/main/Openalex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas requests openpyxl


In [ ]:
import pandas as pd
import requests
import json

# Function to query OpenAlex API based on a person's name
def query_openalex(first_name, last_name):
    base_url = "https://api.openalex.org/authors"
    params = {
        "search": f"{first_name} {last_name}",
        "per_page": 1  # Limit the number of results to 1
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
          # Check if 'count' key exists in 'meta' and if it's greater than 0
        if 'count' in data['meta'] and data['meta']['count'] > 0:
            # If OpenAlex returns results, return the first one
            author = data['results'][0]
            return {
                'OpenAlexID': author['id'],
                'Affiliation': author['affiliation'] if 'affiliation' in author else None,
                'AffiliationID': author['affiliation_id'] if 'affiliation_id' in author else None
            }
        else:
            return None  # No match found
    else:
        print(f"Error querying OpenAlex for {first_name} {last_name}: {response.status_code}")
        return None

# Load the data from the provided Excel file
df = pd.read_excel("ays_faculty.xlsx")

# Prepare lists to store matched and unmatched results
matched = []
unmatched = []

# Loop through the faculty names and query OpenAlex
for index, row in df.iterrows():
    first_name = row['First Name']
    last_name = row['Last Name']

    # Query OpenAlex API
    result = query_openalex(first_name, last_name)

    if result:
        # If a match is found, append the OpenAlex data to the row and add to matched list
        row['OpenAlexIdentifier'] = result['OpenAlexID']
        row['Affiliation'] = result['Affiliation']
        row['AffiliationID'] = result['AffiliationID']
        matched.append(row)
    else:
        # If no match is found, add to unmatched list
        unmatched.append(row)

# Convert the matched and unmatched lists to DataFrames
matched_df = pd.DataFrame(matched)
unmatched_df = pd.DataFrame(unmatched)

# Save the results to CSV or JSON as needed
matched_df.to_csv('matched_faculty.csv', index=False)
unmatched_df.to_csv('unmatched_faculty.csv', index=False)

# Optionally, save the data as JSON
matched_df.to_json('matched_faculty.json', orient='records', lines=True)
unmatched_df.to_json('unmatched_faculty.json', orient='records', lines=True)

print("Process completed. Matched and unmatched faculty saved.")


Process completed. Matched and unmatched faculty saved.


In [ ]:
from google.colab import drive

# This will unmount the drive if it was previously mounted
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

# Remount the drive
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
All changes made in this colab session should now be visible in Drive.
Mounted at /content/drive
